## Importing Libraries

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
df=pd.read_csv('test.csv',encoding='utf-8')

C:\Users\sankh\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263

In [3]:
df.head()

,Unnamed: 0,title,author,text,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 737,Unnamed: 738,Unnamed: 739,Unnamed: 740,Unnamed: 741,Unnamed: 742,Unnamed: 743,Unnamed: 744,Unnamed: 745,Unnamed: 746
0,NaN,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df['label'].value_counts().plot(kind='pie',explode=[0,0.1],figsize=(6,6),autopct='%1.2f%%')
plt.ylabel('Fake vs Not Fake')
plt.legend(['Not Fake','Fake'])
plt.show()

KeyError: 'label'

In [ ]:
# get the independent features
x = df.drop('label',axis=1)

In [ ]:
x.head()

In [ ]:
# get dependent features
y=df['label']
y.head()

In [ ]:
y.value_counts()

In [ ]:
 # droping nan values

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df = df.dropna()

In [ ]:
df.shape

In [ ]:
# make a copy so that original data is not disturbed
messages=df.copy()

In [ ]:
df.head(10)

In [ ]:
# because after droping the nan values some indexes might be lost
messages.reset_index(inplace=True)

In [ ]:
messages.head(10)

## Text Preprocessing

In [ ]:
# remove all stopwords and special characters
import nltk
nltk.download('punkt')

In [ ]:
import re

In [ ]:

nltk.download('stopwords')
  

In [ ]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
 corpus

In [ ]:
# Applying Countvectorizer
# Creating the bag of words model
# max features is that from all the words we take the most 5000 frequently occuring words
# ngram_range is it will take the combintion of words in this range
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,ngram_range=(1,3))
x = cv.fit_transform(corpus).toarray()

In [ ]:
x.shape

In [ ]:
# output feature
y=messages['label']

## Train Test Split

In [ ]:
## Divide the dataset into train and test
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x,y,test_size=0.33,random_state=0)

In [ ]:
#  To check all the feature names that were made using ngram_range
cv.get_feature_names()[:20]

In [ ]:
# all info regarding the count vectorizer
cv.get_params()

In [ ]:
count_df = pd.DataFrame(X_train, columns=cv.get_feature_names())

In [ ]:
# data after converting it into vectors
count_df

In [ ]:
 def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
   
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

## Naive Bayes Algorithm

In [ ]:
from sklearn.naive_bayes import MultinomialNB
NB_classifier =MultinomialNB()

In [ ]:
from sklearn import metrics 
import itertools

In [ ]:
NB_classifier.fit(X_train,y_train)
pred =NB_classifier.predict(X_test)
score = metrics.accuracy_score(y_test,pred)
print('accuracy: %0.3f' % score)
cm = metrics.confusion_matrix(y_test,pred)
plot_confusion_matrix(cm,classes=['Fake','Real'])

## Multinomial Classifier with Hyperparameter

In [ ]:
classifier=MultinomialNB(alpha=0.1)

In [ ]:
previous_score=0
for alpha in np.arange(0,1,0.1):
    sub_classifier=MultinomialNB(alpha=alpha)
    sub_classifier.fit(X_train,y_train)
    y_pred=sub_classifier.predict(X_test)
    score = metrics.accuracy_score(y_test, y_pred)
    if score>previous_score:
        classifier=sub_classifier
    print("Alpha: {}, Score : {}".format(alpha,score))

In [ ]:
# get feature names
feature_names = cv.get_feature_names()

In [ ]:
# The most negative value is the most fake word and the most positive value is most real word
classifier.coef_[0]

In [ ]:
# most real
sorted(zip(classifier.coef_[0],feature_names),reverse=True)[:20]

In [ ]:
# most fake
sorted(zip(classifier.coef_[0],feature_names))[:20]

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

In [ ]:
import pickle

In [ ]:
saved_model=pickle.dumps(NB_classifier)

In [ ]:
modelfrom_pickle = pickle.loads(saved_model) 

In [ ]:
y_pred=modelfrom_pickle.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

In [ ]:
import joblib

In [ ]:
joblib.dump(NB_classifier,'pickle.pkl')

In [ ]:
joblib.dump(cv,'transform.pkl')

In [ ]:
def new_review(new_review):
    
    new_review = new_review
    new_review = re.sub('[^a-zA-Z]', ' ', new_review)
    new_review = new_review.lower()
    new_review = new_review.split()
    ps = PorterStemmer()
    all_stopwords = stopwords.words('english')
    all_stopwords.remove('not')
    new_review = [ps.stem(word) for word in new_review if not word in   set(all_stopwords)]
    new_review = ' '.join(new_review)
    new_corpus = [new_review]
    new_X_test = cv.transform(new_corpus).toarray()
    new_y_pred = modelfrom_pickle.predict(new_X_test)
    
    return new_y_pred
new_review = new_review(str(input("Enter new review...")))

if new_review[0]==1:
    
    print("Fake")
    
else :
    
    print("Not Fake")